In [1]:
import nltk
from nltk.corpus import *

In [2]:
nltk.corpus.words.words()

['A',
 'a',
 'aa',
 'aal',
 'aalii',
 'aam',
 'Aani',
 'aardvark',
 'aardwolf',
 'Aaron',
 'Aaronic',
 'Aaronical',
 'Aaronite',
 'Aaronitic',
 'Aaru',
 'Ab',
 'aba',
 'Ababdeh',
 'Ababua',
 'abac',
 'abaca',
 'abacate',
 'abacay',
 'abacinate',
 'abacination',
 'abaciscus',
 'abacist',
 'aback',
 'abactinal',
 'abactinally',
 'abaction',
 'abactor',
 'abaculus',
 'abacus',
 'Abadite',
 'abaff',
 'abaft',
 'abaisance',
 'abaiser',
 'abaissed',
 'abalienate',
 'abalienation',
 'abalone',
 'Abama',
 'abampere',
 'abandon',
 'abandonable',
 'abandoned',
 'abandonedly',
 'abandonee',
 'abandoner',
 'abandonment',
 'Abanic',
 'Abantes',
 'abaptiston',
 'Abarambo',
 'Abaris',
 'abarthrosis',
 'abarticular',
 'abarticulation',
 'abas',
 'abase',
 'abased',
 'abasedly',
 'abasedness',
 'abasement',
 'abaser',
 'Abasgi',
 'abash',
 'abashed',
 'abashedly',
 'abashedness',
 'abashless',
 'abashlessly',
 'abashment',
 'abasia',
 'abasic',
 'abask',
 'Abassin',
 'abastardize',
 'abatable',
 'abate

In [3]:
import nltk
from nltk.corpus import brown
from nltk import WordNetLemmatizer
from math import log 
wnl=WordNetLemmatizer()

genre=['adventure', 'belles_lettres', 'editorial', 'fiction', 'government', 'hobbies',
'humor', 'learned', 'lore', 'mystery', 'news', 'religion', 'reviews', 'romance',
'science_fiction']
sents=gutenberg.sents(gutenberg.fileids())+brown.sents(categories=genre)
words=gutenberg.words(gutenberg.fileids())+brown.words(categories=genre)

_Fdist = nltk.FreqDist([wnl.lemmatize(w.lower()) for w in words])

_Sents = [[wnl.lemmatize(j.lower()) for j in i] for i in sents]

def p(x):
       return _Fdist[x]/float(len(_Fdist))

def pxy(x,y):
       return (len(list(filter(lambda s :  x in s and y in s ,_Sents)))+1)/ float(len(_Sents) )

def pmi(x,y):
       return  log(pxy(x,y)/(p(x)*p(y)),2) 

In [4]:
import numpy as np
import pandas as pd
training=open("../Desktop/training.txt","r")
validation=open("../Desktop/validation.txt","r")
train_list=training.readlines()
val_list=validation.readlines()
columns=["word1","word2","feature","label"]
training_df=pd.DataFrame(columns=columns)
validation_df=pd.DataFrame(columns=columns)
words=[]
for i in range (len(val_list)):
    w=val_list[i].split(",")
    if w[2][-1]=='?':
        w[2]=w[2][:-1]
    validation_df.loc[i]=[w[0],w[1],w[2],w[3][0]] 
    words.append(w[0])
    words.append(w[1])
    words.append(w[2])
for i in range (len(train_list)):
    w=train_list[i].split(",")
    training_df.loc[i]=[w[0],w[1],w[2],w[3][0]] 
    words.append(w[0])
    words.append(w[1])
    words.append(w[2])

In [5]:
def cal(w1,w2):
    if p(w1)==0:
        w1=wnl.lemmatize(w1)
    if p(w2)==0:
        w2=wnl.lemmatize(w2)
    if p(w1)==0 or p(w2)==0:
        return 0
    else:
        return pmi(w1,w2)


In [6]:
def add_pmi(training_df):
    pmi12=[]
    pmi13=[]
    pmi23=[]
    for index,row in training_df.iterrows():
        
        res12=cal(row[0],row[1])
        res13=cal(row[0],row[2])
        res23=cal(row[1],row[2])
        pmi12.append(res12)
        pmi13.append(res13)
        pmi23.append(res23)
        
    training_df['pmi12']=pmi12
    training_df['pmi13']=pmi13
    training_df['pmi23']=pmi23
    

In [7]:

add_pmi(training_df)

In [83]:
from nltk.corpus import wordnet as wn


def count_feature(word1,wf):
    w1=wn.synsets(word1)
    wds=[]
    for i in w1:
        wds.append(i.definition())
    wds=[wnl.lemmatize(j.lower()).split() for j in wds]
    wds=sum(wds,[])
    return wds.count(wnl.lemmatize(wf))+wds.count(wf)
count_feature('pot','lid')

def add_count(df):
    count13=[]
    count23=[]
    for index,row in df.iterrows():
        count13.append(count_feature(row[0],row[2]))
        count23.append(count_feature(row[1],row[2]))
    df['count13']=count13
    df['count23']=count23
add_count(training_df)


In [92]:
training_df

,word1,word2,feature,label,pmi12,pmi13,pmi23,count13,count23
0,coconut,rice,infested,0,6.962443,10.354761,8.284371,0,0
1,ostrich,emu,cloves,0,0.000000,9.547406,0.000000,0,0
2,bathtub,toilet,barrier,0,7.962443,7.310366,5.239977,0,0
3,pan,skillet,lid,1,7.921801,4.277945,8.032832,0,0
4,pumpkin,spinach,snake,0,11.506764,6.615993,6.615993,0,0
5,crow,crane,flies,1,4.589226,2.018477,2.931014,0,0
6,jeans,shirt,bug,0,4.160989,6.391286,6.045512,0,0
7,raisin,cranberry,clasp,0,11.869334,8.547406,11.354761,0,0
8,pillow,bag,shows,0,3.327684,0.092196,-1.837902,0,0
9,tie,blouse,jazz,0,7.336839,1.707482,7.047332,0,0


In [9]:
add_pmi(validation_df)

In [99]:
from sklearn import svm
X=[]
Y=[]
for index, row in training_df.iterrows():
        X.append(row[4:])
        Y.append(row[3])

#clf=svm.SVC()
#clf.fit(X,Y)

from sklearn import tree
clf = tree.DecisionTreeClassifier()

clf.fit(X, Y)
#lr.predict_proba(X_test_std[0,:])

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [91]:
add_count(validation_df)
validation_df

,word1,word2,feature,label,pmi12,pmi13,pmi23,count13,count23
0,sandwiches,breakfast,lunch,1,3.437090,5.332393,2.493674,0,0
1,surfboard,raft,water,0,0.000000,0.000000,0.906851,0,2
2,banana,cucumber,long,0,8.654321,1.057271,0.678759,0,0
3,lambs,cattle,wool,1,-0.066042,2.955020,1.480701,0,0
4,shrimp,spinach,pink,1,11.506764,6.615993,6.615993,0,0
5,bus,taxi,passengers,0,5.969329,3.409902,4.621406,0,1
6,garage,brick,large,1,4.097373,1.133173,0.952601,0,0
7,walkway,alley,walk,0,10.676689,5.273677,2.273677,0,0
8,polyurethane,polyester,material,0,0.000000,0.000000,3.381494,0,0
9,spoon,potato,wood,1,3.899433,0.303824,0.097373,2,0


In [100]:
predicted=[]
correct=0
total=0
count_1=0
count_1_right=0
count_all_1=0

for index,row in validation_df.iterrows():
    pre=clf.predict([row[4:]])  
    predicted.append(pre)
    if pre=="1":
        count_1+=1
        if pre==row[3]:
            count_1_right+=1
    if pre==row[3]:
        correct+=1
    if row[3]=="1":
        count_all_1+=1
    total+=1
#validation_df["predicted"]=predicted
accuracy=correct/total
print(count_1)
print(count_1_right)
print(count_all_1)
precision=count_1_right/count_1
recall=count_1_right/count_all_1
print ("accuracy:",accuracy)
print ("precision:",precision)
print ("recall:",recall)
print ("F1:",2*precision*recall/(precision+recall))
#validation_df[:100][["word1","word2","feature","label","predicted"]]


799
403
1364
accuracy: 0.5014695077149155
precision: 0.5043804755944932
recall: 0.29545454545454547
F1: 0.37263060564031447
